In [30]:
import random

import numpy as np

from Core.PRef import PRef
from BenchmarkProblems.RoyalRoad import RoyalRoad
from Core.get_pRef import get_pRef_from_metaheuristic
from Core.PS import PS, STAR


def get_win_counts(ps: PS, pRef: PRef):
    with_ps, without_ps = pRef.split_by_indexes(pRef.get_indexes_matching_ps(ps))
    
    non_ps_areas = ps.values == STAR
    
    remaining_without_ps = [(row_non_ps, fitness) for row_non_ps, fitness in zip(without_ps.full_solution_matrix[:, non_ps_areas], without_ps.fitness_array)]
    
    assert(type(with_ps) is PRef)
    assert(type(without_ps) is PRef)
    
    wins = 0
    losses = 0
    ties = 0
    for has_to_have, fitness_with in zip(with_ps.full_solution_matrix[:, non_ps_areas], with_ps.fitness_array):
        for row_without, fitness_without in remaining_without_ps:
            if np.array_equal(row_without, has_to_have):
                if fitness_with > fitness_without:
                    wins += 1
                elif fitness_with < fitness_without:
                    losses +=1
                else:
                    ties += 1
     
    if wins+losses == 0:
        return 0
                
    return wins/(wins+losses)

In [1]:

from BenchmarkProblems.RoyalRoad import RoyalRoad
from Core.get_pRef import get_pRef_from_metaheuristic
from Core.PS import PS

problem = RoyalRoad(5)

pRef = get_pRef_from_metaheuristic(problem=problem,
                                   sample_size=10000,
                                   which_algorithm="GA",
                                   unique=True)

In [12]:
to_evaluate = PS.from_string("1011"+"*"*16)
win_rate = get_win_counts(to_evaluate, pRef)
print(win_rate)

In [21]:
def decompose(starting_ps: PS, already_tested: set[PS]):
    
    print(f"{starting_ps = }")
    if starting_ps.fixed_count() < 2:
        return []
    possible_simplifications = [starting_ps.with_unfixed_value(pos) 
                                for pos in starting_ps.get_fixed_variable_positions()]
    possible_simplifications = [simpl for simpl in possible_simplifications
                                if simpl not in already_tested]
    
    print(f"{possible_simplifications = }")
    
    win_rates = [get_win_counts(simpl, pRef) for simpl in possible_simplifications]
    
    print(f"{win_rates = }")
    
    best_win_rate = 1
    winners = [(simpl, win_rate) for simpl, win_rate in zip(possible_simplifications, win_rates) if win_rate == best_win_rate]
    
    already_tested.update(possible_simplifications) # yes it modifies the argument
    def recursive_call(winner):
        return decompose(winner, already_tested)
    
    children_winners = []
    for winner, win_rate in winners:
        children_winners.extend(recursive_call(winner))
    return winners + children_winners

In [22]:
decomposed = decompose(PS.from_FS(pRef.get_best_solution()), set())

print(decomposed)

starting_ps = [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
possible_simplifications = [[* 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], [1 * 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], [1 1 * 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], [1 1 1 * 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], [1 1 1 1 * 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], [1 1 1 1 1 * 1 1 1 1 1 1 1 1 1 1 1 1 1 1], [1 1 1 1 1 1 * 1 1 1 1 1 1 1 1 1 1 1 1 1], [1 1 1 1 1 1 1 * 1 1 1 1 1 1 1 1 1 1 1 1], [1 1 1 1 1 1 1 1 * 1 1 1 1 1 1 1 1 1 1 1], [1 1 1 1 1 1 1 1 1 * 1 1 1 1 1 1 1 1 1 1], [1 1 1 1 1 1 1 1 1 1 * 1 1 1 1 1 1 1 1 1], [1 1 1 1 1 1 1 1 1 1 1 * 1 1 1 1 1 1 1 1], [1 1 1 1 1 1 1 1 1 1 1 1 * 1 1 1 1 1 1 1], [1 1 1 1 1 1 1 1 1 1 1 1 1 * 1 1 1 1 1 1], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 * 1 1 1 1 1], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 * 1 1 1 1], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 * 1 1 1], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 * 1 1], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 * 1], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 *]]
win_rates = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

KeyboardInterrupt: 

In [34]:


import itertools

smallest_perfect_ps = []
n = pRef.search_space.amount_of_parameters
for ones in itertools.combinations(range(n), r=4):
    ps_values = np.full(shape=n, fill_value=STAR)
    ps_values[list(ones)] = 1
    ps = PS(ps_values)
    win_rate = get_win_counts(ps, pRef)
    if win_rate != 1:
        print(ps)
        smallest_perfect_ps.append(ps)

KeyboardInterrupt: 